In [62]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [63]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [64]:
# convert the input number to a list of 64 bits.
# this is the input to the model

def int_to_64bit_list(number):
    binary_str = format(number, '064b')
    return [int(bit) for bit in binary_str]

ints = [x for x in range(1, 1_000_000)]

input_data = [int_to_64bit_list(x) for x in ints]

In [65]:
# this is just a helper class to load the data into the model
class Data(Dataset):
    def __init__(self, inputs):
        self.inputs = inputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return torch.tensor(self.inputs[idx], dtype=torch.float32)

In [66]:
# this is the machine learning model
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.encoder = nn.Sequential(
          nn.Linear(64, 64 * 5), # 64 bits in
          nn.ReLU(),
          nn.Linear(64 * 5, 64),
          nn.ReLU(),
          nn.Linear(64, 32) # 1 value out
        )

        self.decoder = nn.Sequential(
          nn.Linear(32, 64), # 1 value in
          nn.ReLU(),
          nn.Linear(64, 64 * 2),
          nn.ReLU(),
          nn.Linear(64 * 2, 64) # 64 bits out
        )

    # this is the forward pass. it takes the input and returns the output
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [67]:
# create the dataset and the loader
training_dataset = Data(input_data)
training_loader = DataLoader(training_dataset, batch_size=512, shuffle=True)

# create model and send it to the device, the mac gpu.
model = Model().to(device)

# mean squared error loss and the optimizer
loss = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # learning rate and momentum are hyperparameters that can be tuned

# train the model for 50 iterations
for epoch in range(100):

    # iterate over the data
    for data in training_loader:
        inputs = data
        inputs = inputs.to(device)
        # zero the gradients
        optimizer.zero_grad()

        # run the data through the model, we need to send it to the device as well
        out = model(inputs)
        loss_size = loss(out, inputs)
        loss_size.backward() # backpropagation
        optimizer.step() # update the weights

    print(f"Loss: {loss_size}")
    
    # save the model
torch.save(model.state_dict(), f"model")

Loss: 0.07750330865383148
Loss: 0.07397493720054626
Loss: 0.06241390481591225
Loss: 0.05228934809565544
Loss: 0.04308055713772774
Loss: 0.032657381147146225
Loss: 0.022448968142271042
Loss: 0.014088207855820656
Loss: 0.011028928682208061
Loss: 0.008152117021381855
Loss: 0.007061397656798363
Loss: 0.001991485245525837
Loss: 0.0001838746393332258
Loss: 0.00015590351540595293
Loss: 0.00011025762796634808
Loss: 0.00011811965669039637
Loss: 8.140669524436817e-05
Loss: 7.858265598770231e-05
Loss: 7.720560097368434e-05
Loss: 7.663007272640243e-05
Loss: 6.443342135753483e-05
Loss: 5.719350519939326e-05
Loss: 6.0171314544277266e-05
Loss: 4.7612484195269644e-05
Loss: 5.06916840095073e-05
Loss: 4.899179839412682e-05
Loss: 4.946672197547741e-05
Loss: 5.116333340993151e-05
Loss: 4.046065441798419e-05
Loss: 4.583126792567782e-05
Loss: 4.997431460651569e-05
Loss: 4.8293448344338685e-05
Loss: 3.6191759136272594e-05
Loss: 3.5905228287447244e-05
Loss: 2.9918017389718443e-05
Loss: 3.528290835674852e-05
L

In [90]:
checkpoint = torch.load(f"model", weights_only=False)
model = Model().to(device)
model.load_state_dict(checkpoint)

val = torch.tensor(int_to_64bit_list(100), dtype=torch.float32).to(device)

output = model.encoder(val)

res = model.decoder(output)

# convert values to binary output. 0.5 was a starting point.
binary_tensor = (res >= 0.5).int().tolist()


bit_string = ''.join(map(str, binary_tensor))
number = int(bit_string, 2)

assert number == 100